In [ ]:
import numpy as np
from easyvec import Vec3
from interpolation import Interp1d, Interp2d

In [ ]:
opts = {
    'd': ,
    'L': ,
    'm_0': ,
    't_marsh': ,
    'w_marsh': ,
    'P_marsh': 
}

In [ ]:
class Missile3D(object):
    
    @classmethod
    def get_missile(cls, opts):
        """
        Классовый метод создания ракеты со всеми необходимыми аэродинамическими, массо- и тяговременными характеристиками
        arguments: opts {dict} -- словарь с параметрами ракеты
        returns:   экземпляр класса Missile3D {cls}
        """
        @np.vectorize
        def get_m(t):
            if t < t_marsh:
                return m_0 - G_marsh * t
            else:
                return m_0 - w_marsh
            
        @np.vectorize
        def get_P(t):
            if t < t_marsh:
                return P_marsh
            else:
                return 0
        
        @np.vectorize
        def get_J(t):
            # TODO: функция должна возвращать список или нп массив с 3 числами Ix, Iy, Iz
            if t < t_marsh:
                return (m_0 - G_marsh * t) * L**2
            else:
                return (m_0 - w_marsh) * L**2
        
        g       = opts.get('g', 9.80665)
        d       = opts['d']
        L       = opts['L']
        m_0     = opts['m_0']
        t_marsh = opts['t_marsh']
        w_marsh = opts['w_marsh']
        P_marsh = opts['P_marsh']
        
        G_marsh = w_marsh / t_marsh
        
        ts    = np.linspace(0, t_marsh, 100)
        m_itr = Interp1d(ts, get_m(ts))
        J_itr = get_J(ts)
        P_itr = Interp1d(ts, get_P(ts))
        atm_itr = table_atm
        
        df1 = pd.read_csv('data_constants/cya_from_mach.csv', sep = ";")
        df2 = pd.read_csv('data_constants/cx_from_mach_and_alpha.csv', sep = ";", index_col=0)
        arr_alpha = np.array(df2.index)
        arr_mach = df1['Mach']
        arr_cya = df1['Cya']
        arr_cx = df2.to_numpy()

        Cx_itr  = Interp2d(arr_alpha, arr_mach, arr_cx)
        Cya_itr = Interp1d(arr_mach, arr_cya)
        
        missile = cls(g=g, d=d, P_itr=P_itr, m_itr=m_itr, Cx_itr=Cx_itr, Cya_itr=Cya_itr,
                      atm_itr=atm_itr, J_itr=J_itr)
        return missile
    
    def __init__(self, **kwargs):
        """
        Конструктор класса Missile3D
        """
        self.g    = kwargs['g']
        self.d    = kwargs['d']
        self.P_itr   = kwargs['P_itr']
        self.m_itr   = kwargs['m_itr']
        self.Cx_itr  = kwargs['Cx_itr']
        self.Cya_itr = kwargs['Cya_itr']
        self.atm_itr = kwargs['atm_itr']
        self.J_itr   = kwargs['J_itr']
        self.n = 14
        self.state = np.zeros(self.n)
        self.t = 0
        self.mu_omega = 1.0

    @property
    def pos(self, to_numpy=False):
        if to_numpy == False:
            return Vec3(self.state[0], self.state[1], self.state[2])
        else:
            return np.array([self.state[0], self.state[1], self.state[2]])
        
    @property
    def vel(self, to_numpy=False):
        if to_numpy == False:
            return Vec3(self.state[3], self.state[4], self.state[5])
        else:
            return np.array([self.state[3], self.state[4], self.state[5]])

    @property
    def w(self, to_numpy=False):
        if to_numpy == False:
            return Vec3(self.state[6], self.state[7], self.state[8])
        else:
            return np.array([self.state[6], self.state[7], self.state[8]])
    
    @property
    def quaternion(self):
        """
        Действительные числа кватерниона (кватернион как четырехмерный вектор):
        returns: {np.ndarray} -- q=[qw,qx,qy,qz] 
        """
        return np.array([self.state[9], self.state[10], self.state[11], self.state[12]])
    
    def f_system(self, t, y):
        """
        Функция правой части системы ОДУ динамики материального тела в пространстве
        arguments: t {float}      -- время
                   y {np.ndarray} -- вектор состояния системы 
                                     [0, 1, 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]
                                     [x, y, z, vx, vy, vz, wx, wy, wz, qw, qx, qy, qz,  t]                       
        returns: {np.ndarray}     -- dy/dt         
        """
        rho = self.atm_itr(y[1], 3)

        V = np.sqrt(y[3]**2 + y[4]**2 + y[5]**2)
        
        if V < 1e-5:
            F_Cx = 0
            F_Cx_x = 0
            F_Cx_y = 0
            F_Cx_z = 0
        else:
            F_Cx = self.Cx * rho * V**2 / 2
            F_Cx_x = - F_Cx * (y[3] / V)
            F_Cx_y = - F_Cx * (y[4] / V)
            F_Cx_z = - F_Cx * (y[5] / V)
        
        
        J_x, J_y, J_z = self.J 
        F_x = F_Cx_x + # TODO
        F_y = F_Cx_y + # TODO
        F_z = F_Cx_z + # TODO
        
        dy = np.zeros(self.n)

        dy[0] = y[3] # dx/dt = v_x
        dy[1] = y[4] # dy/dt = v_y
        dy[2] = y[5] # dz/dt = v_z
        dy[3] = F_x / self.m_itr(t) - y[5] * y[7] + y[4] * y[8] # dv_x/dt = a_x
        dy[4] = F_y / self.m_itr(t) - y[3] * y[8] + y[5] * y[6] # dv_y/dt = a_y
        dy[5] = F_z / self.m_itr(t) - y[4] * y[6] + y[3] * y[7] # dv_y/dt = a_y
        dy[6] = (M_x + (J_y - J_z) * y[7] * y[8]) / J_x # domega_x/dt = eps_x
        dy[7] = (M_y + (J_z - J_x) * y[8] * y[6]) / J_y # domega_y/dt = eps_y
        dy[8] = (M_z + (J_x - J_y) * y[6] * y[7]) / J_z # domega_z/dt = eps_z
        dy[9] = -1/2 * (y[6] * y[10] + y[7] * y[11] + y[8] * y[12]) # dq_w/dt
        dy[10] = 1/2 * (y[6] * y[9]  - y[7] * y[12] + y[8] * y[11]) # dq_x/dt 
        dy[11] = 1/2 * (y[7] * y[9]  - y[8] * y[10] + y[6] * y[12]) # dq_y/dt
        dy[12] = 1/2 * (y[8] * y[9]  - y[6] * y[11] + y[7] * y[10]) # dq_z/dt
        dy[13] = 1 # dt
        
        return dy
    
    def validate_y(self, y):
        # TODO: проверка углов
        return y 
    
    def step(self, tau=0.1, n=10):
        """
        Метод моделирования динамики ракеты за шаг по времени tau. 
        На протяжении tau управляющее воздействие на ракету постоянно (action).
        Шаг интегирования dt определяется из числа разбиения n временного шага tau.
        Функция меняет внутреннее состояние ракеты state на момент окончания шага.
        arguments: action {int} -- управляющее воздействие на протяжении шага
                   tau {float}  -- длина шага по времени (не путать с шагом интегрирования).
                   n {int}      -- число шагов разбиений интервала времени tau.
        """
        t = self.state[-1]  
        dt = tau / n
        for i in range(n):
            k1 = self.f_system(t, self.state)
            k2 = self.f_system(t + 0.5 * dt, self.validate_y(self.state + 0.5 * dt * k1))
            k3 = self.f_system(t + 0.5 * dt, self.validate_y(self.state + 0.5 * dt * k2))
            k4 = self.f_system(t + dt,       self.validate_y(self.state + dt * k3))
            y = self.validate_y(self.state + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4))
            t += dt
            self.t = t
            self.state = y
            self.state[-1] = t
        
        
    def to_dict(self):
        return {
            't': self.t,
            'pos': Vec3(self.state[0], self.state[1], self.state[2]),
            'vel': Vec3(self.state[3], self.state[4], self.state[5]),
            'q': self.quaternion,
            'w': self.w
        }
    
    def to_numpy(self):
        return np.array(self.state)
    
    def from_dict(self, state_dict):
        # TODO
        pass
    
    def from_numpy(self, state_numpy):
        # TODO
        pass
    
    def get_B(delta_t, A, velA):
        return A.add_vec(velA.mul_num(delta_t / 3))

    def get_C(delta_t, D, velD): 
        return D.sub_vec(velD.mul_num(delta_t / 3))
    
    def get_traject(self, delta_t, pos_trg, vel_trg, n_points=100):
        taus = np.linspace(0, 1, n_points)
        A = Vec3(self.state[0], self.state[1], self.state[2])
        velA = Vec3(self.state[3], self.state[4], self.state[5])
        D = Vec3(pos_trg[0], pos_trg[1], pos_trg[2])
        velD = Vec3(vel_trg[0], vel_trg[1], vel_trg[2])
        B = self.get_B(delta_t, A, velA)
        C = self.get_C(delta_t, D, velD)
        return np.array([
            (1-tau)**3 * A + 3*tau*(1-tau)**2 * B + 3*tau*tau*(1-tau)*C + tau**3 * D
            for tau in taus
        ])
    
    def get_traject_vels(self, delta_t, pos_trg, vel_trg, n_points=100):
        taus = np.linspace(0, 1, n_points)
        A = Vec3(self.state[0], self.state[1], self.state[2])
        velA = Vec3(self.state[3], self.state[4], self.state[5])
        D = Vec3(pos_trg[0], pos_trg[1], pos_trg[2])
        velD = Vec3(vel_trg[0], vel_trg[1], vel_trg[2])
        B = self.get_B(delta_t, A, velA)
        C = self.get_C(delta_t, D, velD)
        return np.array([
            (3*(1-tau)**2*(B-A) + 6*tau*(1-tau)*(C-B) + 3*tau**2*(D-C)) / delta_t
            for tau in taus
        ])
    
    def get_traject_acc(self, delta_t, pos_trg, vel_trg, n_points=100):
        taus = np.linspace(0, 1, n_points)
        A = Vec3(self.state[0], self.state[1], self.state[2])
        velA = Vec3(self.state[3], self.state[4], self.state[5])
        D = Vec3(pos_trg[0], pos_trg[1], pos_trg[2])
        velD = Vec3(vel_trg[0], vel_trg[1], vel_trg[2])
        B = self.get_B(delta_t, A, velA)
        C = self.get_C(delta_t, D, velD)
        return np.array([
            6 * ((tau-1)*(B-A) + (1-2*tau)*(C-B) + tau*(D-C)) / delta_t**2
            for tau in taus
        ])